In [1]:
#uses DotProduct_Systolic_#N - v1.1x
#uses DotProduct_#N_noHR_C#mamcs_A#baseAdderSize - v3.7
#uses DotProduct_#N_noHR_M_A#baseAdderSize - v3.7
#uses RegisteredMultiplier  - v1.0x

In [2]:
import sys
import math
import import_ipynb


In [8]:
#M>=1
M = input('Enter M: ')

#N>=1
N = input('Enter N: ')

#>=1, <=M
PR = input('Enter No Of Parallel Row (empty for 1): ')

M = int(M)
if(M<1):
    M=1

N = int(N)
if(N<1):
    N=1

if not PR:
    PR=1
else:
    PR = int(PR)
    if PR>M:
        PR = M

#MAMCS>=1
MAMCS = input('Enter Multiply-Add Max Component Size (empty for {}): '.format(N))

if not MAMCS:
    MAMCS = N
else:
    MAMCS = int(MAMCS)
    if MAMCS>N:
        MAMCS = N;
    elif(MAMCS<1):
        MAMCS=1

In [9]:
#AdderSize>=2
if MAMCS<N:
    AdderSize = input('Enter maximum size allowed for base adders (empty for 2): ')

In [10]:
RS=math.ceil(M/PR)

In [11]:
if RS>1:
    Enable_Vector_Latch = input('Enable Vector Latch? (empty or 0 to disable, 1 to enable): ')

In [12]:
#IN_WIDTH>=1
IN_WIDTH = input('Enter IN_WIDTH default value (empty for 10): ')

Enter IN_WIDTH default value (empty for 10): 16


In [13]:
#INPUT_REG_DEPTH>=0
INPUT_REG_DEPTH = input('Enter INPUT_REG_DEPTH default value (empty for 1): ')

Enter INPUT_REG_DEPTH default value (empty for 1): 


In [14]:
#MULT_PIPE_DEPTH>=0
MULT_PIPE_DEPTH = input('Enter MULT_PIPE_DEPTH default value (empty for 1): ')

Enter MULT_PIPE_DEPTH default value (empty for 1): 


In [15]:
NumOfComponents = math.ceil(N/MAMCS)
if MAMCS==N:
    AdderSize=0
else:
    if not AdderSize:
        AdderSize=2
    else:
        AdderSize = int(AdderSize)
        if AdderSize<2:
            AdderSize=2
        if AdderSize>NumOfComponents:
            AdderSize=NumOfComponents

if RS==1:
    Enable_Vector_Latch = 0;
else:
    if not Enable_Vector_Latch:
        Enable_Vector_Latch = 0
    else:
        Enable_Vector_Latch = int(Enable_Vector_Latch)

if not IN_WIDTH:
    IN_WIDTH=10
else:
    IN_WIDTH = int(IN_WIDTH)
    if(IN_WIDTH<1):
        IN_WIDTH=1

if not INPUT_REG_DEPTH:
    INPUT_REG_DEPTH=1
else:
    INPUT_REG_DEPTH = int(INPUT_REG_DEPTH)
    if(INPUT_REG_DEPTH<0):
        INPUT_REG_DEPTH=0

if not MULT_PIPE_DEPTH:
    MULT_PIPE_DEPTH=1
else:
    MULT_PIPE_DEPTH = int(MULT_PIPE_DEPTH)
    if(MULT_PIPE_DEPTH<0):
        MULT_PIPE_DEPTH=0

In [16]:
__Print_To_File = 1

if __Print_To_File<=0:
    of=sys.stdout
else:
    if Enable_Vector_Latch>0:
        of=open("./`_{}_{}_{}PR_noHR_C{}_A{}_VL.v".format(M, N, PR, MAMCS, AdderSize), 'w+')
    else:
        of=open("./MatrixVectorMultiply_{}_{}_{}PR_noHR_C{}_A{}_NVL.v".format(M, N, PR, MAMCS, AdderSize), 'w+')

In [17]:
print("`timescale 1ns / 1ps\n", file=of)
if Enable_Vector_Latch>0:
    print("module MatrixVectorMultiply_{}_{}_{}PR_noHR_C{}_A{}_VL".format(M, N, PR, MAMCS, AdderSize), file=of)
else:
    print("module MatrixVectorMultiply_{}_{}_{}PR_noHR_C{}_A{}_NVL".format(M, N, PR, MAMCS, AdderSize), file=of)
print("#(", file=of)
print("parameter IN_WIDTH = {},".format(IN_WIDTH), file=of)
print("parameter INPUT_REG_DEPTH = {},".format(INPUT_REG_DEPTH), file=of)
print("parameter MULT_PIPE_DEPTH = {}".format(MULT_PIPE_DEPTH), file=of)
print(")(", file=of)
print("input clk, reset, enable,", file=of)
lgRS=math.ceil(math.log2(RS))
if lgRS<=1:
    print("output reg rowSetInNo = 0,", file=of)
else:
    print("output reg [{}:0] rowSetInNo = 0,".format(lgRS-1), file=of)
print("input inReady,", file=of)
for j in range(PR):
    print("input signed [IN_WIDTH-1:0] ", end='', file=of)
    for i in range(N):
        print("AE{}R{}, ".format(i, j), end='', file=of)
    print(file=of)
print("input signed [IN_WIDTH-1:0] ", end='', file=of)
for i in range(N):
    print("VE{}, ".format(i), end='', file=of)
print(file=of)
ar=RS*PR-M
ME = PR-ar-1
if ME==0:
    st1="E0"
else:
    st1="E0toE{}".format(ME)
print("output ", end='', file=of)
print(st1, end='', file=of)
print("Ready,", file=of)
if ar==0:
    print("output reg NUCReady = 0, //not used", file=of)
else:
    if ME==(PR-2):
        st2="E{}".format(PR-1)
    else:
        st2="E{}toE{}".format(ME+1, PR-1)
    print("output ", end='', file=of)
    print(st2, end='', file=of)
    print("Ready,", file=of)
if lgRS<=1:
    print("output reg rowSetOutNo = {},".format(RS-1), file=of)
else:
    print("output reg [{}:0] rowSetOutNo = {},".format(lgRS-1, RS-1), file=of)
lgn=math.ceil(math.log2(N))
al=-1+lgn
if al<0:
    print("output signed [(2*IN_WIDTH){}:0] ".format(al), end='',file=of)
elif al==0:
    print("output signed [(2*IN_WIDTH):0] ", end='',file=of)
else:
    print("output signed [(2*IN_WIDTH)+{}:0] ".format(al), end='',file=of)
for i in range(PR):
    print("E{}, ".format(i), end='',file=of)
print(file=of)
print("output early", end='', file=of)
print(st1, end='', file=of)
print("Ready,", file=of)
if ar==0:
    print("output reg NUearlyCReady = 0 //not used", file=of)
else:
    print("output early", end='', file=of)
    print(st2, end='', file=of)
    print("Ready", file=of)
print(");\n", file=of)

In [18]:
if Enable_Vector_Latch>0:
    print("reg signed [IN_WIDTH-1:0] ", end='', file=of)
    for i in range(N-1):
        print("SVE{}, ".format(i), end='', file=of)
    print("SVE{};".format(N-1), file=of)
    print(file=of)

In [19]:
if(ar>0): #RS*PR > M, RS>=2
    print("wire inReadyC = (rowSetInNo!={}) & inReady;".format(RS-1), file=of)
print(file=of)
for i in range (PR):
    if N==1:
        print("RegisteredMultiplier", file=of)
    elif MAMCS==N:
        print("DotProduct_Systolic_{}".format(N), file=of)
    elif MAMCS==1:
        print("DotProduct_{}_noHR_M_A{}".format(N, AdderSize), file=of)  
    else:
        print("DotProduct_{}_noHR_C{}_A{}".format(N, MAMCS, AdderSize), file=of)
    print("#( .IN_WIDTH(IN_WIDTH), .INPUT_REG_DEPTH(INPUT_REG_DEPTH), .MULT_PIPE_DEPTH(MULT_PIPE_DEPTH) )", file=of)
    print("DP_{} (clk, reset, enable,".format(i), file=of)
    if(PR-i<=ar):
        print("inReadyC,", file=of)
    else:
        print("inReady,", file=of)
    for j in range(N):
        print("AE{}R{}, ".format(j, i), end='', file=of)
    print(file=of)
    if Enable_Vector_Latch>0:
        for j in range(N):
            print("SVE{}, ".format(j), end='', file=of)
    else:
        for j in range(N):
            print("VE{}, ".format(j), end='', file=of)        
    print(file=of)
    if i==0:
        print(st1+"Ready,", file=of)
    elif i==PR-ar:
        print(st2+"Ready,", file=of)
    else:
        print("DPOutReady{}, //not used".format(i), file=of)
    print("E{},".format(i), file=of)
    if i==0:
        print("early"+st1+"Ready", file=of)
    elif i==PR-ar:
        print("early"+st2+"Ready", file=of)
    else:
        print("DPearlyOutReady{} //not used".format(i), file=of)
    print(");\n", file=of)

In [20]:
if(RS>1):
    if (Enable_Vector_Latch>0):
        print("reg signed [IN_WIDTH-1:0] ", end='', file=of)
        for i in range(N-1):
            print("LVE{}, ".format(i), end='', file=of)
        print("LVE{};".format(N-1), file=of)
    print("always @(posedge clk) begin", file=of)
    print("\tif(reset) begin", file=of)
    print("\t\trowSetInNo <= 0;", file=of)
    print("\tend", file=of)
    print("\telse if(enable) begin", file=of)
    print("\t\tif(inReady) begin", file=of)
    print("\t\t\tif(rowSetInNo=={}) begin".format(RS-1), file=of)
    print("\t\t\t\trowSetInNo <= 0;", file=of)
    print("\t\t\tend", file=of)
    print("\t\t\telse begin", file=of)
    if RS==2:
        print("\t\t\t\trowSetInNo <= 1;", file=of)    
    else:
        print("\t\t\t\trowSetInNo <= rowSetInNo+1;", file=of)
    print("\t\t\tend", file=of)
    if Enable_Vector_Latch>0:
        print("\t\t\tif(rowSetInNo==0) begin", file=of)
        print("\t\t\t\t", end='', file=of)
        for j in range(N):
            print("LVE{} <= VE{}; ".format(j, j), end='', file=of)
        print(file=of)
        print("\t\t\tend", file=of)
    print("\t\tend", file=of)
    print("\tend", file=of)
    print("end\n", file=of)

In [21]:
if Enable_Vector_Latch>0:
    print("always @(*) begin", file=of)
    print("\tif(rowSetInNo==0) begin", file=of)
    print("\t\t", end='', file=of)
    for j in range(N):
        print("SVE{} = VE{}; ".format(j,j), end='', file=of)
    print(file=of)
    print("\tend", file=of)
    print("\telse begin", file=of)
    print("\t\t", end='', file=of)
    for j in range(N):
        print("SVE{} = LVE{}; ".format(j, j), end='', file=of)
    print(file=of)
    print("\tend", file=of)
    print("end\n", file=of)

In [22]:
if(RS>1):
    print("always @(posedge clk) begin", file=of)
    print("\tif(reset) begin", file=of)
    print("\t\trowSetOutNo <= {};".format(RS-1), file=of)
    print("\tend", file=of)
    print("\telse if(enable) begin", file=of)
    print("\t\tif(early"+st1+"Ready) begin", file=of)
    print("\t\t\tif(rowSetOutNo=={}) begin".format(RS-1), file=of)
    print("\t\t\t\trowSetOutNo <= 0;", file=of)
    print("\t\t\tend", file=of)
    print("\t\t\telse begin", file=of)
    if RS==2:
        print("\t\t\t\trowSetOutNo <= 1;", file=of)
    else:
        print("\t\t\t\trowSetOutNo <= rowSetOutNo+1;", file=of)
    print("\t\t\tend", file=of)
    print("\t\tend", file=of)
    print("\tend", file=of)
    print("end\n", file=of)

In [23]:
print("endmodule", file=of)

In [24]:
if __Print_To_File>0:
    of.close()